# Setting up the simulation and running the server

Just by running the cell below (and wait until the asterisk you see see at the top left of the cell changes to a number between the brackets) you will be able to access to the server Demo_Server using the Nanover App

In [ ]:
import openmm as mm
from openmm import app as app
from openmm import unit as unit
import sys
from nanover.openmm.serializer import serialize_simulation
from nanover.omni import OmniRunner
from nanover.omni.openmm import OpenMMSimulation
from nanover.app import NanoverImdClient
from nanover.app import RenderingSelection
import matplotlib.cm
from nanover.mdanalysis import frame_data_to_mdanalysis

# Loading the files to simulate and set up the simulation
pdb = app.PDBFile('openmm/openmm_files/output_filename.pdb')
forcefield = app.ForceField('amber99sb.xml', 'tip3p.xml')
system = forcefield.createSystem(
    pdb.topology,
    nonbondedMethod=app.PME,
    nonbondedCutoff=1 * unit.nanometer,
    constraints=app.HBonds,
    removeCMMotion=False,
) 

integrator = mm.LangevinIntegrator(
    300 * unit.kelvin,
    1 / unit.picosecond,
    0.002 * unit.picoseconds,
)
simulation = app.Simulation(pdb.topology, system, integrator)
simulation.context.setPositions(pdb.positions)

simulation.minimizeEnergy()
simulation.step(100)
pdb.topology

# Create the OpenMMSimulation instance
protein_simulation = OpenMMSimulation.from_simulation(simulation)


# Pass the simulation to the runner
imd_runner = OmniRunner.with_basic_server(protein_simulation, name="Demo_Server", port=0)

# Run the simulation
imd_runner.next()

print(f'{imd_runner.app_server.name}: serving at {imd_runner.app_server.address}:{imd_runner.app_server.port}')

from nanover.app import NanoverImdClient

client = NanoverImdClient.connect_to_single_server(port=imd_runner.app_server.port)
client.subscribe_to_frames()
client.wait_until_first_frame();
# Begin tracking updates to the shared multiplayer values
client.subscribe_multiplayer()

def hide_root(show: bool = True):
    root_selection = client.root_selection
    with root_selection.modify():
        root_selection.set_particles(generate_mdanalysis_selection("protein and not type H",
                                                        v=False))
        root_selection.hide = show
        root_selection.interaction_method = 'single'

def generate_mdanalysis_selection(selection: str, v: bool=False):
    # Crear el objeto Universe a partir del primer frame
    universe = frame_data_to_mdanalysis(client.first_frame)
    
    # Seleccionar los átomos según la selección
    selected_atoms = universe.select_atoms(selection)
    
    # Obtener los residuos únicos (para evitar duplicados) asociados a los átomos seleccionados
    selected_residues = selected_atoms.residues
    
    # Si verbose es True, imprimir los nombres de los residuos
    if v:
        for residue in selected_residues:
            print(f"Residuo: {residue.resname}, Resid ID: {residue.resid}")
    
    # Retornar los índices de los átomos seleccionados
    idx_array = selected_atoms.indices
    
    # Retornar los índices como enteros
    return list(map(int, idx_array))



## Run this cell below to apply cartoon style

>**NOTE:**
>There is still a bug related with the renderization of the protein, when it has more than one chain, in cartoon it renders just one of them

In [ ]:
protein = client.create_selection("all", [])

with protein.modify():
    protein.hide = False
    protein.set_particles(generate_mdanalysis_selection("all"))
    protein.renderer = 'cartoon'

In [8]:
imd_runner.close()